# Removing Sensitive Information from Panoramic Images

## By Rohan and Vivian
## Group 25


# 1. Loading the model, weights and Images

### 1.1 Mount the drive

The drive contains images, labels and all the necessary config files

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
# drive.flush_and_unmount()

Mounted at /content/gdrive


### 1.2 Import Libraries

In [2]:
# Setup
import numpy
import matplotlib.pyplot as plt
import sys
from google.colab import files
import os
import shutil


from six.moves import urllib    
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

# for imshow
from google.colab.patches import cv2_imshow

# Torch Libraries
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision.datasets as dset
import torchvision.utils as vutils
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.transforms.functional as F

from IPython.display import Image, clear_output  # to display images

## 1.3 Installing dependencies

1. Installing the repo from YOLOv5 official git repo
2. Installing Wandb for visualising purposes

In [3]:
# Add the Yolov5 git
def add_yolo_git():
  !git clone https://github.com/ultralytics/yolov5  # clone repo
  %cd yolov5
  %pip install -qr requirements.txt  # install dependencies

add_yolo_git()  
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

!pip install wandb
import wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 5760, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 5760 (delta 1), reused 3 (delta 0), pack-reused 5750
Receiving objects: 100% (5760/5760), 8.23 MiB | 21.79 MiB/s, done.
Resolving deltas: 100% (3935/3935), done.
/content/yolov5
     |████████████████████████████████| 645kB 9.8MB/s 
     |████████████████████████████████| 2.1MB 7.5MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
     |████████████████████████████████| 163kB 37.1MB/s 
     |████████████████████████████████| 133kB 30.7MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=5d6b1a253f1689560269ea5077cc9a14d9f357dd986ea230965de04283f89de5
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtoo

In [4]:
# To ensure it runs over an appropriate processor
CUDA = torch.cuda.is_available()
print("CUDA :: ", CUDA)
device = torch.device("cuda" if CUDA else "cpu")

CUDA ::  False


### 1.4 Load the existing model and the weights

In [5]:
# Weights dir
# model_dir = "./gdrive/MyDrive/DL_TUD/small_final-20210319T094746Z-001/small_final/weights/last.pt"
# model_dir='/content/gdrive/MyDrive/DL_TUD/small_final-20210319T094746Z-001/small_final/weights/best.pt'
# medium_model_dir = ""
# model = pytorch.
# model = torch.load(model_dir, encoding='ascii')
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
# model = model.load_state_dict()
# model.eval()

# model = torch.hub.load(model_dir, 'best.pt', source = 'local')

### 1.4 Transfer the images to Train/Test/Validation Folders

In [6]:
import os
import shutil

def transfer_images(file_dir, old_dir_images, new_dir_images, old_dir_labels, new_dir_labels):
  """
  Transfer train/test/val Images
  """
  f = open(file_dir, 'r+')

  for line in f.readlines():
    image_name = line.split("/")[-1]
    image_name = image_name.replace("\n", "")
    label_name = image_name.replace(".jpg", ".txt")
    print(image_name)
    if (os.path.isfile(old_dir_images + image_name) and os.path.isfile(old_dir_labels + label_name)):
        shutil.copy(old_dir_images + image_name, new_dir_images + image_name)
        shutil.copy(old_dir_labels + label_name, new_dir_labels + label_name)
        print("success!")

In [7]:
# TRAIN
# text file that contains the list of directories of all training images 
file_dir = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/train.txt"

# Images
old_dir_images = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/images_small/"
new_dir_images = "/content/gdrive/MyDrive/DL_TUD/data/images/train/" 

# Labels
old_dir_labels = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/labels/"
new_dir_labels = "/content/gdrive/MyDrive/DL_TUD/data/labels/train/"

# Run this function if you want to tranfer, please ignore of the images are already in the said folder
# transfer_images(file_dir, old_dir_images, new_dir_images, old_dir_labels, new_dir_labels)

In [8]:
# VALIDATION

file_dir_val = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/val.txt"

# Images
old_dir_images = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/images_small/"
new_dir_images = "/content/gdrive/MyDrive/DL_TUD/data/images/val/" 

# Labels
old_dir_labels = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/labels/"
new_dir_labels = "/content/gdrive/MyDrive/DL_TUD/data/labels/val/"

# transfer_images(file_dir_val, old_dir_images, new_dir_images, old_dir_labels, new_dir_labels)

In [9]:
# TEST

file_dir_val = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/test.txt"

# Images
old_dir_images = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/images_small/"
new_dir_images = "/content/gdrive/MyDrive/DL_TUD/data/images/test/" 

# Labels
old_dir_labels = "/content/gdrive/MyDrive/DL_TUD/training_set/training_set/labels/"
new_dir_labels = "/content/gdrive/MyDrive/DL_TUD/data/labels/test/"

# transfer_images(file_dir_val, old_dir_images, new_dir_images, old_dir_labels, new_dir_labels)

# 2. Train the model

In [10]:
# def train_model(data_yaml, cfg_yaml):
#   # !python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache
#   !python3 train.py --adam --img 2048 --batch 8 --epochs 20 --save_period 4 --data $data_yaml --cfg $cfg_yaml --weights '' --name /content/gdrive/MyDrive/DL_TUD/yolov5s_results_actual_50 
#   print("Successfully trained")

In [11]:
def train_model(data_yaml, cfg_yaml, out_name, epoch=30, weight_dir=''):
  """
  Trains the model for the given data from:
  data config file: train and val directories
  network architecture file: head, backbone and others
  """
# !python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache

  # No pretrained weights
  if weight_dir == '':
    !python3 train.py --adam --img 2048 --batch 8 --epochs $epoch --save_period 4 --data $data_yaml --cfg $cfg_yaml --weights '' --name $out_name
  # Pretrained weights, incase the running stops in the middle  
  else:
    !python3 train.py --adam --img 2048 --batch 8 --epochs $epoch --save_period 4 --data $data_yaml --cfg $cfg_yaml --weights $weight_dir --name $out_name
  print("Successfully trained...I guess")

In [12]:
# Yaml that mentions the class names, train and val directories
data_yaml_train = '/content/gdrive/MyDrive/DL_TUD/data/data_custom.yaml'

# Yaml model file for small model
cfg_yaml_s_train = '/content/gdrive/MyDrive/DL_TUD/models/custom_yolov5s.yaml'
# Yaml model file for medium model
cfg_yaml_m_train = '/content/gdrive/MyDrive/DL_TUD/models/custom_yolov5m.yaml'

# Save trained output, log files and plots to this directory
out_name_train = '/content/gdrive/MyDrive/DL_TUD/yolov5s_results_actual_50'


In [13]:
# Run Wandb offline 
# !wandb offline

# code = 'wandb offline'
# !$code

In [14]:
# train_model(data_yaml_train, cfg_yaml_s_train,out_name_train,epoch=50,weight_dir='')  

## 2.2 Hyperparam check



In [15]:
def train_model_hyp(data_yaml, cfg_yaml, hyp_yaml, out_name, epoch=30, weight_dir=''):
  """
  Trains the model for the given data from:
  data config file: train and val directories
  network architecture file: head, backbone sections
  hyperparameter file;
  """
# !python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --nosave --cache

  if weight_dir == '':
    !python3 train.py --adam --img 2048 --batch 4 --epochs $epoch --save_period 4 --data $data_yaml --cfg $cfg_yaml --weights '' --name $out_name --hyp $hyp_yaml 
  else:
    !python3 train.py --adam --img 2048 --batch 4 --epochs $epoch --save_period 4 --data $data_yaml --cfg $cfg_yaml --weights $weight_dir --name $out_name --hyp $hyp_yaml
  print("Successfully trained...I guess")

In [43]:
# Hyperparameter config file directories, uncomment for what you like
# hyp = 'per_0005' #0.0005
# hyp = 'per_001' #0.001
# hyp = 'shear_10' #10 degrees
# hyp = 'shear_20' #20 degrees
# hyp = 'mix_up_05' #0.5 mixup probability
hyp = 'mix_up_1' #1 mixup probability
# The initial weights to begin (if stops in the middle of training, outta nowhere)

start_weight_dir = '/content/gdrive/MyDrive/DL_TUD/yolov5s_results_' + hyp + '/weights/last.pt'
# For yaml

hyp_yaml = '/content/gdrive/MyDrive/DL_TUD/models/custom_hyp_' + hyp + '.yaml'
# Saves the output to this directory

out_name = '/content/gdrive/MyDrive/DL_TUD/yolov5s_results_' + hyp
# number of epochs

epoch = 12

In [44]:
# Train with a different variation of hyperparameter!

# train_model_hyp(data_yaml_train, cfg_yaml_s_train, hyp_yaml, out_name, epoch, start_weight_dir)

github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-7-g1487bc8 torch 1.8.1+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=True, artifact_alias='latest', batch_size=4, bbox_interval=-1, bucket='', cache_images=False, cfg='/content/gdrive/MyDrive/DL_TUD/models/custom_yolov5s.yaml', data='/content/gdrive/MyDrive/DL_TUD/data/data_custom.yaml', device='', entity=None, epochs=12, evolve=False, exist_ok=False, global_rank=-1, hyp='/content/gdrive/MyDrive/DL_TUD/models/custom_hyp_mix_up_1.yaml', image_weights=False, img_size=[2048, 2048], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='/content/gdrive/MyDrive/DL_TUD/yolov5s_results_mix_up_1', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='/content/gdrive/MyDrive/DL_TUD/yolov5s_results_mix_up_18', save_period=4, single_cls=False, sync_bn=False, total_batch_size=4, upload_dataset=False, weights='/content/gdrive/M

In [16]:
!wandb sync /content/yolov5/yolov5/wandb/offline-run-20210415_102745-13ks1b9a

Usage: wandb sync [OPTIONS] [PATH]...
Try 'wandb sync --help' for help.

Error: Invalid value for '[PATH]...': Path '/content/yolov5/yolov5/wandb/offline-run-20210415_102745-13ks1b9a' does not exist.


# 3. Loading the Trained Model and evaluating the test images

In [17]:
# def run_model_detect(weight_dir, img_source, save_img_dir, conf=0.25):
#   !python detect.py --weights $weight_dir --conf $conf --source $img_source

# testing our images

def run_detect_images(weight_dir, img_source, save_img_dir, conf=0.25):
  """
  Function to Detect in the images and store it in an folder  
  weight_dir: directory of the trained weights, it is a pt file
  img_source: directory of the images to evaluate
  save_img_dir: results will be stored in this directory
  conf: this is the confidence threshold
  """
  # !python detect.py --weights /content/gdrive/MyDrive/DL_TUD/yolov5s_results_50/weights/best.pt --conf 0.25 --source /content/gdrive/MyDrive/DL_TUD/Pano_images/resizedIMG_20210414_202223.jpg
  !python detect.py --weights $weight_dir --conf $conf --source $img_source --project $save_img_dir

In [21]:
# """
#   Function to Detect in the images and store it in an expt folder
# """
# def run_model_detect(weight_dir, img_source, conf=0.25):
#   !python detect.py --weights $weight_dir --conf $conf --source $img_source

In [22]:
## Run the detection for all the images
# run_model_detect(weight_dir_custom, img_source_custom)
img_source='/content/gdrive/MyDrive/DL_TUD/data/images/test'
weight_dir_test='/content/gdrive/MyDrive/DL_TUD/yolov5s_results_50/weights/best.pt'
save_img_dir = '/content/gdrive/MyDrive/DL_TUD/Test_Detected'

# Show time!
run_detect_images(weight_dir_test, img_source, save_img_dir)

## 3.2 Infer the model

Let us see how it performs on the images taken in Amsterdam, from the test set!

In [23]:
Image(filename= '/content/yolov5/runs/detect/exp4/TMX7315120208-000061_pano_0001_000205.jpg', width=600)

FileNotFoundError: ignored

## 3.3 Testing it on our images! 

We took a panoramic image in our neighborhood to find out what is like for the model to classify on an unseen data! Here we go!

In [ ]:
weight_dir_eval = '/content/gdrive/MyDrive/DL_TUD/yolov5s_results_50/weights/best.pt'
img_source = '/content/gdrive/MyDrive/DL_TUD/Pano_images/'
save_img_dir = '/content/gdrive/MyDrive/DL_TUD/Pano_detected/'

# Show time!
run_detect_images(weight_dir_eval, img_source, save_img_dir)

In [ ]:
# Let us see the resultant image!
from google.colab.patches import cv2_imshow
# Image(filename=save_img_dir + 'exp/Street View 360 1 first.jpg', width=600)
Image(filename='/content/gdrive/MyDrive/DL_TUD/Pano_detected/exp/IMG_20210414_202223.jpg', width=600)
# cv2_imshow(save_img_dir + 'exp/Street View 360 1 first.jpg')
# /content/gdrive/MyDrive/DL_TUD/Pano_images/Street View 360 1 first.jpg

In [ ]:
# Incase you want to resize the image

def resize_img(img_source_dir):
  for filename in os.listdir(img_source_dir):
    img = cv2.imread(filename)

    desired_dim = 2048
    factor = img.shape[1] / desired_dim
    width = int(img.shape[1] / factor)
    height = int(img.shape[0] / factor)
    dim = (width, height)

    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    print('Original Dimensions : ',img.shape) 
    print('Resized Dimensions : ',resized.shape)

    cv2.imwrite(filename + '_resized.jpg', resized)


In [ ]:
# torch.cuda.empty_cache()
# torch.cuda.memory_summary()

In [ ]:
# REMOVE FOLDERS

def remove_stuff(rem_dir):
  !rm -rf $rem_dir

In [19]:
# !nvidia-smi
# !sudo fuser -v /dev/nvidia*
# !sudo kill -9 59
# torch.cuda.empty_cache()
# !sudo nvidia-smi --gpu-reset -i 0

Thu Apr 15 10:22:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------